## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-10-13-02-38 +0000,bbc,Children feared among three dead in Channel cr...,https://www.bbc.com/news/articles/ckgy0n4d99jo...
1,2025-09-10-13-00-00 +0000,wsj,If the Supreme Court rules in favor of the pre...,https://www.wsj.com/economy/trade/tariff-case-...
2,2025-09-10-13-00-00 +0000,missionlocal,Basket case,https://missionlocal.org/2025/09/sf-photos-bas...
3,2025-09-10-12-59-16 +0000,wapo,"What to know as Poland invokes NATO Article 4,...",https://www.washingtonpost.com/world/2025/09/1...
4,2025-09-10-12-57-53 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/09/10/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2294/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
62,trump,58
121,israel,21
155,protests,19
17,new,18
219,qatar,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
213,2025-09-09-22-19-57 +0000,wapo,Israeli strike targeting Hamas in Qatar ‘does ...,https://www.washingtonpost.com/world/2025/09/0...,144
180,2025-09-10-00-03-37 +0000,nypost,Trump asks EU to levy up to 100% tariffs on Ch...,https://nypost.com/2025/09/09/world-news/trump...,117
166,2025-09-10-01-14-36 +0000,nyt,"With Qatar Attack, Israel Again Leaves Trump i...",https://www.nytimes.com/2025/09/09/us/politics...,111
149,2025-09-10-02-45-20 +0000,nypost,Court rules that Lisa Cook can keep job as Fed...,https://nypost.com/2025/09/09/us-news/court-ru...,108
67,2025-09-10-10-00-00 +0000,nypost,"Trump will ‘take over the Fed,’ ex-economic ad...",https://nypost.com/2025/09/10/us-news/trump-wi...,106


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
213,144,2025-09-09-22-19-57 +0000,wapo,Israeli strike targeting Hamas in Qatar ‘does ...,https://www.washingtonpost.com/world/2025/09/0...
296,81,2025-09-09-18-28-45 +0000,nypost,French President Emmanuel Macron names loyalis...,https://nypost.com/2025/09/09/world-news/frenc...
315,65,2025-09-09-17-00-25 +0000,nyt,Protests Against Nepal’s Social Media Ban Show...,https://www.nytimes.com/2025/09/09/world/asia/...
176,61,2025-09-10-00-36-34 +0000,nypost,Poland’s airspace ‘repeatedly violated’ by Rus...,https://nypost.com/2025/09/09/world-news/ukrai...
66,58,2025-09-10-10-00-00 +0000,nypost,Lisa Cook should be kicked out of Fed’s Septem...,https://nypost.com/2025/09/10/us-news/lisa-coo...
1,54,2025-09-10-13-00-00 +0000,wsj,If the Supreme Court rules in favor of the pre...,https://www.wsj.com/economy/trade/tariff-case-...
11,46,2025-09-10-12-26-25 +0000,nypost,Jeffrey Epstein’s assistant reveals names of l...,https://nypost.com/2025/09/10/us-news/jeffrey-...
38,44,2025-09-10-11-33-00 +0000,wsj,"Rep. Jerry Nadler of New York, age 78, said he...",https://www.wsj.com/politics/democrats-retirem...
192,39,2025-09-09-23-10-04 +0000,latimes,Kennedy commission child health report ignores...,https://www.latimes.com/science/story/2025-09-...
255,39,2025-09-09-20-54-54 +0000,nypost,Wife of late Sen. John McCain facing calls to ...,https://nypost.com/2025/09/09/us-news/cindy-mc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
